In [1]:
%load_ext autotime

time: 0 ns (started: 2022-09-23 04:14:30 -05:00)


# MODULES

In [37]:
import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import numpy as np
import math

import pandas as pd

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

time: 16 ms (started: 2022-09-23 04:14:35 -05:00)


# CLASESS

## Vec2d

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

time: 16 ms (started: 2022-09-23 04:14:34 -05:00)


## Explorers

In [4]:
class Explorers(object):
    
    # This class substitutes BM trajectory and CREW Trajectory
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
        
    def trace_trajectories(self, n_traj = 1, traj_type= "BM_2d", n_steps =1000, speeds = [6], crw_exponents = [0.5]):
        """
            Generates the trajectories of the explorers according to their type
            Arguments: 
                traj_type:
                n_steps:
                speed:
                n_traj:
                crw_exponents:
          """
        
        trajectories_df = pd.DataFrame(columns=['x','y','traj'])
        

        for j in range(n_traj):
            
            velocity = Vec2d(speeds[j], 0)
            
            
            traj_aux_array = np.ones((n_steps,3))*[self.x,self.y,0]
            traj_aux_array[0,2] = j

            for i in range(1,n_steps):
                crw_ex = lambda : crw_exponents[j] if len(crw_exponents) > 1 else i
                turn_angle = self.generate_angle(traj_type, crw_ex())
                
                velocity = velocity.rotated(turn_angle)

                traj_aux_array[i] = traj_aux_array[i-1,:]+[velocity.x,velocity.y,0]
                traj_aux_array[i,2] = j

                
            temp_df = pd.DataFrame(data = traj_aux_array, columns=['x','y','traj'])
            
            trajectories_df = pd.concat([trajectories_df,temp_df], ignore_index=True)
          
        return trajectories_df
    
    
    
    
    # angle choice 
    def generate_angle(self,traj_type, exponent = 0.6):
            
        if traj_type == "BM_2d":
            angle = np.random.uniform(low=-np.pi, high=np.pi)
        
        if traj_type == "BM_nsew":
            angle = np.random.choice([0,np.pi/2, np.pi, 3*np.pi])
        
        if traj_type == "CRW":
            angle = wrapcauchy.rvs(exponent)
            
            
        return angle

time: 16 ms (started: 2022-09-23 04:14:34 -05:00)


## Levy Flight

In [5]:
class LFlyTrajectory(object):
    
    def __init__(self, x,y):
        self.x=0
        self.y=0
        
        
    def levy_walk(self, alpha = 1, beta = 1, loc = 6 , speed = 3, samples = 100000):
        # Init velocity vector

        velocity = Vec2d(speed,0)

        # Init df
        LW_df = pd.DataFrame(columns=['x','y','traj'])
        lw_3d = np.array([[0,0,0]])
        aux = np.array([[0,0,0]])


        i = 1
        while i < samples:
            # get random n_steps form levy distribution
            step_size = levy_stable.rvs(alpha, beta, loc)
            step_size = int(np.ceil(abs(step_size)))

            theta = wrapcauchy.rvs(c=0.7, loc=0)

            # update velocity
            velocity = velocity.rotated(theta)

            for j in range(step_size):
                aux[0,:] = lw_3d[i-1,:]+[velocity.x,velocity.y,0]
                lw_3d = np.r_[lw_3d,aux]

                i+=1

        temp_df = pd.DataFrame(data = lw_3d, columns=['x','y','traj'])
        LW_df = pd.concat([LW_df,temp_df], ignore_index=True)

        return LW_df;

time: 0 ns (started: 2022-09-23 04:14:34 -05:00)


## Metrics

### Path lenght

In [6]:
class TrajectoryMetrics():
    
    
    def get_path_lengt(self, trajectories, traj_df):
        
        path_lenghts = pd.DataFrame(columns=['distance','traj'])
            
        for i in range(n_traj):
            aux_df= trajectories.loc[trajectories.traj == i, ['x','y']]

            #Calculate lenghts for given trajectory
            euc_dis = np.array([get_euclidean_distance(aux_df.iloc[i-1], aux_df.iloc[i]) for i in range(1,aux_df.shape[0])])


            temp_df = pd.DataFrame(data = np.cumsum(euc_dis), columns = ['distance'])

            trajectory_id = np.array([i for j in range(1,aux_df.shape[0])])
            temp_df['traj'] = trajectory_id

            path_lenghts = pd.concat([path_lenghts,temp_df], ignore_index=True)
                
            
        return path_lenghts
    
    
    
    def get_trajectories_msd(self, trajectories, traj_df):
        
        msd_df = pd.DataFrame(columns=['MSD','traj'])
        
        for i in range(n_traj):
            aux_df= bm_2d_df.loc[bm_2d_df.traj == i, ['x','y']]
            msd = np.empty(shape=(0))

            for tau in range(1,aux_df.shape[0]):
                BM_msd = get_msd(tau,aux_df)

                msd = np.append(msd,BM_msd)

            temp_df = pd.DataFrame(data = msd, columns = ['MSD'])

            trajectory_id = np.array([i for j in range(1,aux_df.shape[0])])
            temp_df['traj'] = trajectory_id

            msd_df = pd.concat([msd_df,temp_df], ignore_index=True)
                
        
        return msd_df
    
    
    
    def get_ta_distribution(self, traj_df):
        
        ta_dist = np.empty(shape=(0))

    
        for index, row in traj_df[1:-1].iterrows():
            
            turning_angle = get_turning_angle(traj_df.iloc[index-1],traj_df.iloc[index],traj_df.iloc[index+1])
            ta_dist = np.append(ta_dist,turning_angle)
        
        return ta_dist
    
    
    
    def get_sl_distribution(self, traj_df):
        
            ta = self.get_ta_distribution(traj_df)
            
            ta_df= pd.DataFrame(data = ta, columns=['TA_Levy'])
            
            
            sl_array = np.empty(shape=(0))
            
            for group, elements in ta_df.groupby((ta_df['TA_Levy'].shift()!= ta_df['TA_Levy']).cumsum()):
                if elements.shape[0]>1:
                    sl_array = np.append(sl_array,elements.shape[0]+1)
                    
            return sl_array

time: 0 ns (started: 2022-09-23 04:14:34 -05:00)


## Auxiliar functions

### Euclidean distance

$d_E(p,q)=\sqrt{(p_x-q_x)^2+(p_y-q_y)^2}$

In [7]:
def get_euclidean_distance(p,q):
    """
        Arguments:
            p: [x,y] values for the starting point
            q: [x,y] values for the ending point
    """
  
    distance = np.sqrt(np.square(p[0]-q[0]) + np.square(p[1]-q[1]))

    return distance

time: 0 ns (started: 2022-09-23 04:14:34 -05:00)


### Mean Square Displacement


$MSD = \frac{1}{N-n} \sum \limits_{i=1}^{N-n}(\vec{r}_{i+n}-\vec{r}_i)^2 \quad\quad n=1,...,N-1$
<br><br>

$MSD = \frac{1}{N-n}\sum \limits_{i=1}^{N-n}{d_E(p,q)}^2 \quad\quad n=1,...,N-1$

In [8]:
def get_msd(tau,path):
    """
      Arguments:
        tau:
        path:
    """

    square_displacement = 0 

    for i in range(tau,path.shape[0],1):
        square_displacement += np.square(get_euclidean_distance(path.iloc[i-tau], path.iloc[i]))

    msd= (1/(path.shape[0]-tau))*square_displacement

    return msd

time: 0 ns (started: 2022-09-23 04:14:34 -05:00)


### Turning Angle

$
tan(\phi)=\frac{|\vec{p}\times\vec{q}|}{\vec{p}\cdot\vec{q}}
$

In [9]:
def get_turning_angle(a,b,c,round_to_zero = False):
    """
      Arguments:
        a: coordinates for p vector's tail
        b: coordinates p vector's head / q vector's tail
        c: coordinates q vector's head
        round_to_zero: if true checks if value is close enough to zero to be take as it
     """

    p = np.subtract(b,a)
    q = np.subtract(c,b)

  
    pq_cross = np.cross(p,q)
    pq_scalar = np.dot(p,q)

    phi_angle = np.arctan2(pq_cross,pq_scalar)

    if round_to_zero:
        if  count_like_zero(phi_angle):
            phi_angle = 0
      
    return phi_angle

time: 0 ns (started: 2022-09-23 04:14:34 -05:00)


### Custom round

In [10]:
def count_like_zero(number):
    """
        Arguments:
            number: 
        Return: True if -0.009 <= number <= 0.009 False otherwise
    """
    is_zero = False

    if  ((number >= -0.009) & (number <= 0)) | ((number >= 0) & (number <= 0.009)):
        is_zero = True

    return is_zero

time: 0 ns (started: 2022-09-23 04:14:34 -05:00)


### Is Negligible turn

In [11]:
def drop_turn(angle_i, angle_j):
    """
      Arguments:
        angle_i:
        angle_j: 
      Return: True if difference between angles is less than 0.001 False otherwise
      """
    is_dropable = False
    
    if (angle_i > 0) & (angle_j > 0): 
        rest = abs(angle_i - angle_j)
        if abs(angle_i - angle_j) < 0.001:
            is_dropable = True

    return is_dropable

time: 0 ns (started: 2022-09-23 04:14:34 -05:00)


# PLOT FUNCTIONS

## Plot Trajectories

In [12]:
def plot_trajectories(trajectories_df, n_trajectories = 1, title = "", line_name_prefix="", line_name_subfix = [""]):
    
    fig_3d = go.Figure()

    for i in range(n_trajectories):
        
        subfix = lambda : line_name_subfix[i] if len(line_name_subfix) > 1 else i
        trajectory_name = line_name_prefix +": " + str(subfix())

        fig_3d.add_scatter3d(
                x = trajectories_df.loc[trajectories_df['traj']==i,'x'], 
                y = trajectories_df.loc[trajectories_df['traj']==i,'y'], 
                z = trajectories_df.index,
                marker = dict(size=2),
                line = dict(width=2),
                mode = 'lines',
                name = trajectory_name,
                showlegend = True)

    
    fig_3d.update_layout(
            title_text = title,
            autosize = False,
            width = 800,
            height = 800,
            scene_camera = dict(eye = dict(x=0, y=0, z=2.5)),
            scene = dict(
                xaxis = dict(title = 'x_pos (mm'),
                yaxis = dict(title = 'y_pos (mm)'),
                zaxis = dict(title = 'time', nticks = 20)
            ))

    fig_3d.show()

time: 0 ns (started: 2022-09-23 04:14:34 -05:00)


## Plot Metrics

In [13]:
def plot_metrics(metrics_df, n_trajectories = 1, title = "", line_name_prefix="", line_name_subfix = [""]):
    
    fig_2d = go.Figure()
    
    for i in range(n_trajectories):
        
        subfix = lambda : line_name_subfix[i] if len(line_name_subfix) > 1 else i
        trajectory_name = line_name_prefix +": " + str(subfix())

        fig_2d.add_scatter( 
                    x = metrics_df.index,
                    y = metrics_df.loc[metrics_df['traj']==i,'MSD'], 
                    marker = dict(size=0.5),
                    line = dict(width=3),
                    mode = 'lines',
                    name = trajectory_name,
                    showlegend = True)
        
    fig_2d.update_layout(
            title_text = title,
            autosize = False)

    fig_2d.show()

time: 16 ms (started: 2022-09-23 04:14:34 -05:00)


# PRUEBAS

## Trajectory controls

In [121]:
css = '''
.bk.controls_main {
    max-width: 400vw !important;
    height: 880px !important;
    border-radius: 15px;
}

.bk.min_h_control {
    min-height: 50px !important;
    top: 50;
}


.bk.hide_blocks {
}

.bk.show_blocks {
    min-height: 200px !important;
}

'''

pn.extension(raw_css=[css])

time: 63 ms (started: 2022-09-23 04:45:30 -05:00)


In [62]:
traj_type = pnw.RadioButtonGroup(
    name='Trajectory type', 
    options=['Brownian motion','Correlated Random Walk', 'Levy Flight'], 
    button_type = 'primary')

time: 0 ns (started: 2022-09-23 04:15:43 -05:00)


In [63]:
traj_type

RadioButtonGroup(button_type='primary', name='Trajectory type', options=['Brownian motion', ...], value='Brownian motion')

time: 31 ms (started: 2022-09-23 04:15:44 -05:00)


## Basic Controls

In [64]:

n_traj = pnw.IntSlider(name='Numer of trajectories', value=1, step=1, start=1, end=4)

s_x = pnw.IntInput(name='X Starting position',value=0, step=1, start=-100, end=100)
s_y = pnw.IntInput(name='Y Starting position',value=0, step=1, start=-100, end=100)

speed = pnw.IntInput(name='Speed',value=6, step=1, start=1, end=10)

n_steps = pnw.IntSlider(name='Numer of steps', value=1000, step=1000, start=1000, end=100000)

time: 16 ms (started: 2022-09-23 04:15:45 -05:00)


In [65]:
s_pos_controls = pn.Row(s_x,s_y)

time: 0 ns (started: 2022-09-23 04:15:45 -05:00)


In [66]:
step_speed_controls =  pn.Row(speed,n_steps)

time: 15 ms (started: 2022-09-23 04:15:46 -05:00)


In [67]:
basic_controls = pn.Column(n_traj,s_pos_controls, step_speed_controls)

time: 0 ns (started: 2022-09-23 04:15:46 -05:00)


In [68]:
basic_controls

Column
    [0] IntSlider(end=4, name='Numer of trajectories', start=1, value=1)
    [1] Row
        [0] IntInput(end=100, name='X Starting position', start=-100)
        [1] IntInput(end=100, name='Y Starting position', start=-100)
    [2] Row
        [0] IntInput(end=10, name='Speed', start=1, value=6)
        [1] IntSlider(end=100000, name='Numer of steps', start=1000, step=1000, value=1000)

time: 109 ms (started: 2022-09-23 04:15:46 -05:00)


## CRW Controls

In [69]:
crw_e_1 = pnw.FloatInput(name='c exponent',value=0.6, step=0.1, start=0.1, end=0.9)
crw_e_2 = pnw.FloatInput(name='c exponent for trajectory no. 2',value=0.6, step=0.1, start=0.1, end=0.9, visible = False)
crw_e_3 = pnw.FloatInput(name='c exponent for trajectory no. 3',value=0.6, step=0.1, start=0.1, end=0.9, visible = False)
crw_e_4 = pnw.FloatInput(name='c exponent for trajectory no. 4',value=0.6, step=0.1, start=0.1, end=0.9, visible = False)

time: 16 ms (started: 2022-09-23 04:15:47 -05:00)


In [70]:
apply_crwe_to_all = pnw.RadioButtonGroup(name='crwe_apply', options=['YES', 'NO'], button_type = 'default')

time: 16 ms (started: 2022-09-23 04:15:47 -05:00)


In [71]:
s_pos_controls = pn.Column('####Apply same exponent to all trajectories?',apply_crwe_to_all)

time: 15 ms (started: 2022-09-23 04:15:48 -05:00)


In [72]:
s_pos_controls

Column
    [0] Markdown(str)
    [1] RadioButtonGroup(name='crwe_apply', options=['YES', 'NO'], value='YES')

time: 47 ms (started: 2022-09-23 04:15:48 -05:00)


In [73]:
exponents = pn.Column(pn.Row(crw_e_1,crw_e_2),pn.Row(crw_e_3,crw_e_4))

time: 16 ms (started: 2022-09-23 04:15:49 -05:00)


In [74]:
exponents

Column
    [0] Row
        [0] FloatInput(end=0.9, name='c exponent', start=0.1, value=0.6)
        [1] FloatInput(end=0.9, name='c exponent for t..., start=0.1, value=0.6, visible=False)
    [1] Row
        [0] FloatInput(end=0.9, name='c exponent for t..., start=0.1, value=0.6, visible=False)
        [1] FloatInput(end=0.9, name='c exponent for t..., start=0.1, value=0.6, visible=False)

time: 79 ms (started: 2022-09-23 04:15:49 -05:00)


In [75]:
CRW_controls = pn.Column('### CRW exponents',s_pos_controls,exponents, css_classes = ['hide_blocks'])

time: 16 ms (started: 2022-09-23 04:15:49 -05:00)


## Levy controls

In [122]:
levy_alpha = pnw.FloatInput(name='alpha',value=1, step=0.1, start=0, end=2, css_classes = ['min_h_control'])
levy_beta = pnw.FloatInput(name='beta',value=1, step=0.1, start=0, end=1, css_classes = ['min_h_control'])
levy_loc = pnw.IntSlider(name='loc', value=3, step=1, start=0, end=10, css_classes = ['min_h_control'])

time: 16 ms (started: 2022-09-23 04:45:36 -05:00)


In [123]:
Levy_controls = pn.Column(
    '### Levy values',pn.Row(levy_alpha,levy_beta, levy_loc),
css_classes = ['hide_blocks'])

time: 0 ns (started: 2022-09-23 04:45:43 -05:00)


## ALL

In [124]:
controls_layout = pn.Column(
    traj_type,basic_controls,CRW_controls, Levy_controls, 
     background = 'WhiteSmoke',
css_classes = ['controls_main'])

time: 0 ns (started: 2022-09-23 04:45:44 -05:00)


In [125]:
controls_layout

Column(background='WhiteSmoke', css_classes=['controls_main'])
    [0] RadioButtonGroup(button_type='primary', name='Trajectory type', options=['Brownian motion', ...], value='Levy Flight')
    [1] Column
        [0] IntSlider(end=4, name='Numer of trajectories', start=1, value=1)
        [1] Row
            [0] IntInput(end=100, name='X Starting position', start=-100)
            [1] IntInput(end=100, name='Y Starting position', start=-100)
        [2] Row
            [0] IntInput(end=10, name='Speed', start=1, value=6)
            [1] IntSlider(end=100000, name='Numer of steps', start=1000, step=1000, value=1000)
    [2] Column(css_classes=['hide_blocks'], visible=False)
        [0] Markdown(str)
        [1] Column
            [0] Markdown(str)
            [1] RadioButtonGroup(name='crwe_apply', options=['YES', 'NO'], value='YES')
        [2] Column
            [0] Row
                [0] FloatInput(end=0.9, name='c exponent', start=0.1, value=0.6)
                [1] FloatInput(end=0.9, name='c exponent for t..., start=0.1, value=0.6, visible=False)
            [1] Row
                [0] FloatInput(end=0.9, name='c exponent for t..., start=0.1, value=0.6, visible=False)
                [1] FloatInput(end=0.9, name='c exponent for t..., start=0.1, value=0.6, visible=False)
    [3] Column(css_classes=['hide_blocks'])
        [0] Markdown(str)
        [1] Row
            [0] FloatInput(css_classes=['min_h_control'], end=2, name='alpha', start=0, value=1)
            [1] FloatInput(css_classes=['min_h_control'], end=1, name='beta', start=0, value=1)
            [2] IntSlider(css_classes=['min_h_control'], end=10, name='loc', value=3)

time: 218 ms (started: 2022-09-23 04:45:44 -05:00)


In [112]:
@pn.depends(traj_type)
def display_Radio_Button(traj_type):
    if traj_type == "Correlated Random Walk":
        #CRW_controls.css_classes = ['show_blocks']
        #Levy_controls.css_classes = ['hide_blocks']
        CRW_controls.visible = True 
        Levy_controls.visible = False 

    else:
        if traj_type == "Levy Flight":
            #CRW_controls.css_classes = ['hide_blocks']
            #Levy_controls.css_classes = ['show_blocks']
            CRW_controls.visible = False 
            Levy_controls.visible = True 

        else:
            #CRW_controls.css_classes = ['hide_blocks']
            #Levy_controls.css_classes = ['hide_blocks']
            CRW_controls.visible = False 
            Levy_controls.visible = False 


time: 0 ns (started: 2022-09-23 04:40:14 -05:00)


In [113]:
pn.Row(display_Radio_Button)

Row
    [0] ParamFunction(function)

time: 32 ms (started: 2022-09-23 04:40:15 -05:00)


In [114]:
plots = pn.Row('# Trajectory', "# Metrcis")

time: 16 ms (started: 2022-09-23 04:40:17 -05:00)


In [95]:
pn.Row(controls_layout, plots)

Row
    [0] Column(background='WhiteSmoke', css_classes=['controls_main'])
        [0] RadioButtonGroup(button_type='primary', name='Trajectory type', options=['Brownian motion', ...], value='Brownian motion')
        [1] Column
            [0] IntSlider(end=4, name='Numer of trajectories', start=1, value=1)
            [1] Row
                [0] IntInput(end=100, name='X Starting position', start=-100)
                [1] IntInput(end=100, name='Y Starting position', start=-100)
            [2] Row
                [0] IntInput(end=10, name='Speed', start=1, value=6)
                [1] IntSlider(end=100000, name='Numer of steps', start=1000, step=1000, value=1000)
        [2] Column(css_classes=['hide_blocks'], visible=False)
            [0] Markdown(str)
            [1] Column
                [0] Markdown(str)
                [1] RadioButtonGroup(name='crwe_apply', options=['YES', 'NO'], value='YES')
            [2] Column
                [0] Row
                    [0] FloatInput(end=0.9, name='c exponent', start=0.1, value=0.6)
                    [1] FloatInput(end=0.9, name='c exponent for t..., start=0.1, value=0.6, visible=False)
                [1] Row
                    [0] FloatInput(end=0.9, name='c exponent for t..., start=0.1, value=0.6, visible=False)
                    [1] FloatInput(end=0.9, name='c exponent for t..., start=0.1, value=0.6, visible=False)
        [3] Column(background='WhiteSmoke', css_classes=['hide_blocks'], visible=False)
            [0] Markdown(str)
            [1] Row
                [0] FloatInput(end=2, name='alpha', start=0, value=1)
                [1] FloatInput(end=1, name='beta', start=0, value=1)
            [2] IntSlider(end=10, name='loc', value=3)
    [1] Row
        [0] Markdown(str)
        [1] Markdown(str)

time: 219 ms (started: 2022-09-23 04:22:27 -05:00)
